In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [24]:
def imshow(title = "Image", image = None, size = 10):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

In [36]:
cap = cv2.VideoCapture("videos/walking.avi")

width = int(cap.get(3))
height = int(cap.get(4))

out = cv2.VideoWriter("videos/optical_flow_walking.avi", cv2.VideoWriter_fourcc("M", "J", "P", "G"), 30, (width, height))
feature_params = dict(maxCorners=100,
                     qualityLevel=0.3,
                     minDistance=7,
                     blockSize=7)
lucas_kanade_params = dict(winSize=(15, 15),
                          maxLevel=2,
                          criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
color = np.random.randint(0, 255, (100, 3))

ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

mask = np.zeros_like(prev_frame)

while True:
    ret, frame = cap.read()
    if ret == True:
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        new_corners, status, errors = cv2.calcOpticalFlowPyrLK(prev_gray,
                                                              frame_gray,
                                                              prev_corners,
                                                              None,
                                                              **lucas_kanade_params)
        good_new = new_corners[status==1]
        good_old = prev_corners[status==1]
    
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = map(int, new.ravel())
            c, d = map(int, old.ravel())
            mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
            frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
    
        img = cv2.add(frame, mask)
    
        out.write(img)
    
        prev_gray = frame_gray.copy()
        prev_corners = good_new.reshape(-1, 1, 2)
    else:
        break

cap.release()
out.release()

In [40]:
cap = cv2.VideoCapture("videos/walking.avi")

width = int(cap.get(3))
height = int(cap.get(4))

out = cv2.VideoWriter("videos/dense_optical_flow_walking.avi", cv2.VideoWriter_fourcc("M", "J", "P", "G"), 30, (width, height))

ret, first_frame = cap.read()
previous_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(first_frame)
hsv[..., 1] = 255

while True:
    ret, frame2 = cap.read()
    if ret == True:
        next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(previous_gray, next,
                                           None, 0.5, 3, 15, 3, 5, 1.2, 0)

        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = angle*(180/(np.pi/2))
        hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        final = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        out.write(final)
        previous_gray = next
    else:
        break
cap.release()
cap.release()